In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

'/srv/idp-radio-1'

In [2]:
import os 
import tensorflow as tf
from pathlib import Path

In [3]:
import cv2

In [4]:
# Run this before loading other dependencies, otherwise they might occupy memory on gpu 0 by default and it will stay that way

# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(device_count={'GPU': 1}, allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 1.0
tf.compat.v1.Session(config=config)


Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:04:00.0, compute capability: 7.5



In [5]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from src.architectures.simple.simple_base import SimpleBaseArchitecture
from src.architectures.adv.guendel19 import densenet
import numpy as np

from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from tensorflow.keras.optimizers import Adam, SGD
from src.architectures.benchmarks.benchmark_definitions import Chexpert_Benchmark, Chestxray14_Benchmark, simple_architecture_experiment, generate_benchmarks, METRICS, SINGLE_CLASS_METRICS, CHEXPERT_COLUMNS, CHESTXRAY14_COLUMNS
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, BinaryCrossentropy, compute_class_weight

In [6]:
#CHEXPERT_BENCHMARKS, CHESTXRAY14_BENCHMARKS = generate_benchmarks (batch_sizes={"Medium": 32}, epoch_sizes= {"Dev": 5,}, crop={"C0": False}, name_suffix="",)



In [7]:
#CHEXPERT_BENCHMARKS.keys() 

In [8]:
#CHESTXRAY14_BENCHMARKS.keys()

In [9]:
#[m.name for m in METRICS]

In [10]:
#CHEXPERT_COLUMNS

In [11]:
!remote_access/get_tunnels.sh

URLs open for the following services:
[('tensorboard', 'http://2ed6ba3ba26a.ngrok.io'), ('jupyterlab', 'http://cf3e4fece84f.ngrok.io')]


In [12]:
#CHEXPERT_BENCHMARKS["BCE_E5_B32_C0"].as_dict()

In [13]:
#chexpert_exp = simple_architecture_experiment(CHEXPERT_BENCHMARKS["BCE_E5_B32_C0"], DenseNet121, CHEXPERT_COLUMNS)

In [14]:
#chexpert_exp.benchmark.as_dict()

In [15]:
#chexpert_exp.run()

In [16]:
columns_12 = ['Enlarged Cardiomediastinum',
                    'Cardiomegaly',
                    'Lung Opacity',
                    'Lung Lesion',
                    'Edema',
                    'Consolidation',
                    'Pneumonia',
                    'Atelectasis',
                    'Pneumothorax',
                    'Pleural Effusion',
                    'Pleural Other',
                    'Fracture']
columns_5 =  ['Cardiomegaly',
                'Edema',
                'Consolidation',
                'Atelectasis',
                'Pleural Effusion']

uzeros = ['Cardiomegaly',
        'Enlarged Cardiomediastinum',
        'Lung Opacity',
        'Lung Lesion',
        'Consolidation',
        'Pneumothorax',
        'Pleural Effusion']

uones = ['Edema',
        'Atelectasis',
        'Fracture',
        'Pleural Other',
        'Pneumonia',]

upsample_factors = {
    "Enlarged Cardiomediastinum": 1,
    "Lung Lesion":1,
    #"Pneumothorax":1,
    #"Pneumonia":1,
    "Pleural Other":2,
    "Fracture":2,
}
transformations_0 = {"hist_equalization":{}}
transformations_1 = { "gaussian_blur":{"kernal_size":3}, "hist_equalization":{}}
transformations_2 = {"unsharp_mask":{"radius":2, "amount":1}}
transformations_3 = {"windowing"}

In [18]:
bce_benchmark = Chexpert_Benchmark (path = Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY")),
                                             name="Chexpert_CWBCE_L1Normed_E3_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_5LR1_LF1_SGD_Upsampled",
                                             classes=columns_12,
                                             train_labels = "train.csv",
                                             test_labels = "test.csv",
                                             nan_replacement = 0, #float("NaN"),
                                             u_enc = [uzeros, uones],
                                             epochs=3,
                                             batch_size=32,
                                             crop = False,
                                             dim=(256, 256),
                                             #loss = BinaryCrossentropy(),
                                             use_class_weights = False,
                                             upsample_factors = upsample_factors,
                                             metrics=METRICS,
                                             single_class_metrics=SINGLE_CLASS_METRICS,
                                             optimizer = SGD(learning_rate=5e-1, clipnorm=1),
                                             lr_factor = 0.1,
                                             augmentation = "affine",
                                             transformations = transformations_2,
                                             split_seed = 6122156,
                                             split_valid_size = 0.05, 
                                             preprocess_input_fn = tf.keras.applications.inception_v3.preprocess_input)

bce_benchmark.loss = WeightedBinaryCrossentropy(bce_benchmark.positive_weights,
                                                bce_benchmark.negative_weights)

In [19]:
bce_benchmark.as_dict()

{'benchmark_name': 'Chexpert_CWBCE_L1Normed_E3_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_5LR1_LF1_SGD_Upsampled',
 'dataset_name': 'chexpert_full',
 'dataset_folder': 'data/chexpert/full',
 'models_dir': 'models',
 'epochs': 3,
 'optimizer': 'SGD',
 'learning_rate': 0.5,
 'lr_factor': 0.1,
 'loss': 'weighted_binary_crossentropy',
 'use_class_weights': False,
 'class_weights': [1.0471136569976807,
  0.8615774512290955,
  0.21405130624771118,
  1.2290253639221191,
  0.3829084038734436,
  1.5654457807540894,
  0.9355764389038086,
  0.3477109670639038,
  1.1680023670196533,
  0.27839189767837524,
  1.1709423065185547,
  0.7992541790008545],
 'positive_weights': [1.0471136569976807,
  0.8615774512290955,
  0.21405130624771118,
  1.2290253639221191,
  0.3829084038734436,
  1.5654457807540894,
  0.9355764389038086,
  0.3477109670639038,
  1.1680023670196533,
  0.27839189767837524,
  1.1709423065185547,
  0.7992541790008545],
 'negative_weights': [0.7336834073066711,
  0.7509576678276062,
 

In [20]:
bce_chexpert_exp = simple_architecture_experiment(bce_benchmark, InceptionV3, bce_benchmark.label_columns)

87916544/87910968 [==============================] - 3s 0us/step


In [21]:
bce_chexpert_exp.train()

Epoch 1/3
3718/8344 [============>.................] - ETA: 19:50 - loss: 0.2745 - auc: 0.6938 - precision: 0.6507 - recall: 0.0502 - f2_score: 0.0613 - binary_accuracy: 0.8243 - accuracy_enlarged_cardiomediastinum: 0.9046 - accuracy_cardiomegaly: 0.8847 - accuracy_lung_opacity: 0.5267 - accuracy_lung_lesion: 0.9162 - accuracy_edema: 0.7526 - accuracy_consolidation: 0.9320 - accuracy_pneumonia: 0.8903 - accuracy_atelectasis: 0.7093 - accuracy_pneumothorax: 0.9120 - accuracy_pleural_effusion: 0.6772 - accuracy_pleural_other: 0.9114 - accuracy_fracture: 0.8748 - auc_enlarged_cardiomediastinum: 0.5862 - auc_cardiomegaly: 0.7407 - auc_lung_opacity: 0.6733 - auc_lung_lesion: 0.6675 - auc_edema: 0.7892 - auc_consolidation: 0.6659 - auc_pneumonia: 0.6511 - auc_atelectasis: 0.6480 - auc_pneumothorax: 0.7183 - auc_pleural_effusion: 0.7961 - auc_pleural_other: 0.7322 - auc_fracture: 0.6571 - precision_enlarged_cardiomediastinum: 0.3617 - precision_cardiomegaly: 0.5402 - precision_lung_opacity: 0

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



8344/8344 [==============================] - ETA: 0s - loss: 0.2667 - auc: 0.7200 - precision: 0.6752 - recall: 0.0724 - f2_score: 0.0879 - binary_accuracy: 0.8271 - accuracy_enlarged_cardiomediastinum: 0.9044 - accuracy_cardiomegaly: 0.8877 - accuracy_lung_opacity: 0.5275 - accuracy_lung_lesion: 0.9169 - accuracy_edema: 0.7600 - accuracy_consolidation: 0.9333 - accuracy_pneumonia: 0.8907 - accuracy_atelectasis: 0.7093 - accuracy_pneumothorax: 0.9130 - accuracy_pleural_effusion: 0.6953 - accuracy_pleural_other: 0.9115 - accuracy_fracture: 0.8758 - auc_enlarged_cardiomediastinum: 0.6149 - auc_cardiomegaly: 0.7773 - auc_lung_opacity: 0.6877 - auc_lung_lesion: 0.6973 - auc_edema: 0.8074 - auc_consolidation: 0.6858 - auc_pneumonia: 0.6749 - auc_atelectasis: 0.6665 - auc_pneumothorax: 0.7561 - auc_pleural_effusion: 0.8224 - auc_pleural_other: 0.7588 - auc_fracture: 0.6904 - precision_enlarged_cardiomediastinum: 0.4475 - precision_cardiomegaly: 0.5658 - precision_lung_opacity: 0.5896 - preci

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



5780/8344 [===================>..........] - ETA: 10:24 - loss: 0.2384 - auc: 0.7987 - precision: 0.7092 - recall: 0.1552 - f2_score: 0.1840 - binary_accuracy: 0.8363 - accuracy_enlarged_cardiomediastinum: 0.9066 - accuracy_cardiomegaly: 0.8982 - accuracy_lung_opacity: 0.5262 - accuracy_lung_lesion: 0.9213 - accuracy_edema: 0.7851 - accuracy_consolidation: 0.9339 - accuracy_pneumonia: 0.8931 - accuracy_atelectasis: 0.7070 - accuracy_pneumothorax: 0.9222 - accuracy_pleural_effusion: 0.7410 - accuracy_pleural_other: 0.9171 - accuracy_fracture: 0.8841 - auc_enlarged_cardiomediastinum: 0.7156 - auc_cardiomegaly: 0.8564 - auc_lung_opacity: 0.7294 - auc_lung_lesion: 0.8124 - auc_edema: 0.8531 - auc_consolidation: 0.7536 - auc_pneumonia: 0.7483 - auc_atelectasis: 0.7131 - auc_pneumothorax: 0.8605 - auc_pleural_effusion: 0.8746 - auc_pleural_other: 0.8599 - auc_fracture: 0.8079 - precision_enlarged_cardiomediastinum: 0.5872 - precision_cardiomegaly: 0.6324 - precision_lung_opacity: 0.7617 - pr

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



8344/8344 [==============================] - ETA: 0s - loss: 0.2376 - auc: 0.8003 - precision: 0.7090 - recall: 0.1580 - f2_score: 0.1871 - binary_accuracy: 0.8367 - accuracy_enlarged_cardiomediastinum: 0.9067 - accuracy_cardiomegaly: 0.8985 - accuracy_lung_opacity: 0.5266 - accuracy_lung_lesion: 0.9213 - accuracy_edema: 0.7856 - accuracy_consolidation: 0.9338 - accuracy_pneumonia: 0.8930 - accuracy_atelectasis: 0.7078 - accuracy_pneumothorax: 0.9225 - accuracy_pleural_effusion: 0.7421 - accuracy_pleural_other: 0.9176 - accuracy_fracture: 0.8850 - auc_enlarged_cardiomediastinum: 0.7173 - auc_cardiomegaly: 0.8574 - auc_lung_opacity: 0.7298 - auc_lung_lesion: 0.8140 - auc_edema: 0.8542 - auc_consolidation: 0.7553 - auc_pneumonia: 0.7502 - auc_atelectasis: 0.7147 - auc_pneumothorax: 0.8626 - auc_pleural_effusion: 0.8755 - auc_pleural_other: 0.8623 - auc_fracture: 0.8108 - precision_enlarged_cardiomediastinum: 0.5895 - precision_cardiomegaly: 0.6330 - precision_lung_opacity: 0.7753 - preci

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



7756/8344 [==========================>...] - ETA: 2:20 - loss: 0.2301 - auc: 0.8156 - precision: 0.7180 - recall: 0.1792 - f2_score: 0.2108 - binary_accuracy: 0.8398 - accuracy_enlarged_cardiomediastinum: 0.9075 - accuracy_cardiomegaly: 0.9016 - accuracy_lung_opacity: 0.5292 - accuracy_lung_lesion: 0.9246 - accuracy_edema: 0.7915 - accuracy_consolidation: 0.9335 - accuracy_pneumonia: 0.8922 - accuracy_atelectasis: 0.7102 - accuracy_pneumothorax: 0.9254 - accuracy_pleural_effusion: 0.7465 - accuracy_pleural_other: 0.9235 - accuracy_fracture: 0.8923 - auc_enlarged_cardiomediastinum: 0.7398 - auc_cardiomegaly: 0.8662 - auc_lung_opacity: 0.7342 - auc_lung_lesion: 0.8320 - auc_edema: 0.8623 - auc_consolidation: 0.7709 - auc_pneumonia: 0.7653 - auc_atelectasis: 0.7254 - auc_pneumothorax: 0.8818 - auc_pleural_effusion: 0.8806 - auc_pleural_other: 0.8897 - auc_fracture: 0.8387 - precision_enlarged_cardiomediastinum: 0.6023 - precision_cardiomegaly: 0.6431 - precision_lung_opacity: 0.8591 - pre

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [22]:
bce_chexpert_exp.evaluate()

  1/234 [..............................] - ETA: 32s - loss: 0.1203 - auc: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - f2_score: 0.0000e+00 - binary_accuracy: 0.9167 - accuracy_enlarged_cardiomediastinum: 1.0000 - accuracy_cardiomegaly: 1.0000 - accuracy_lung_opacity: 1.0000 - accuracy_lung_lesion: 1.0000 - accuracy_edema: 1.0000 - accuracy_consolidation: 1.0000 - accuracy_pneumonia: 1.0000 - accuracy_atelectasis: 1.0000 - accuracy_pneumothorax: 1.0000 - accuracy_pleural_effusion: 1.0000 - accuracy_pleural_other: 1.0000 - accuracy_fracture: 0.0000e+00 - auc_enlarged_cardiomediastinum: 0.0000e+00 - auc_cardiomegaly: 0.0000e+00 - auc_lung_opacity: 0.0000e+00 - auc_lung_lesion: 0.0000e+00 - auc_edema: 0.0000e+00 - auc_consolidation: 0.0000e+00 - auc_pneumonia: 0.0000e+00 - auc_atelectasis: 0.0000e+00 - auc_pneumothorax: 0.0000e+00 - auc_pleural_effusion: 0.0000e+00 - auc_pleural_other: 0.0000e+00 - auc_fracture: 0.0000e+00 - precision_enlarged_cardiomediastinum: 0.0000e+00 - 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1

234/234 [==============================] - 13s 55ms/step - loss: 0.3028 - auc: 0.7704 - precision: 0.6180 - recall: 0.1007 - f2_score: 0.1210 - binary_accuracy: 0.8130 - accuracy_enlarged_cardiomediastinum: 0.5385 - accuracy_cardiomegaly: 0.7436 - accuracy_lung_opacity: 0.4615 - accuracy_lung_lesion: 0.9744 - accuracy_edema: 0.8718 - accuracy_consolidation: 0.8632 - accuracy_pneumonia: 0.9573 - accuracy_atelectasis: 0.6667 - accuracy_pneumothorax: 0.9573 - accuracy_pleural_effusion: 0.7906 - accuracy_pleural_other: 0.9658 - accuracy_fracture: 0.9658 - auc_enlarged_cardiomediastinum: 0.5603 - auc_cardiomegaly: 0.7771 - auc_lung_opacity: 0.9031 - auc_lung_lesion: 0.9657 - auc_edema: 0.9208 - auc_consolidation: 0.9023 - auc_pneumonia: 0.7705 - auc_atelectasis: 0.8030 - auc_pneumothorax: 0.7873 - auc_pleural_effusion: 0.9280 - auc_pleural_other: 0.9270 - auc_fracture: 0.0000e+00 - precision_enlarged_cardiomediastinum: 0.6667 - precision_cardiomegaly: 0.9000 - precision_lung_opacity: 0.0000

{'report': '                            precision    recall  f1-score   support\n\nEnlarged Cardiomediastinum       0.67      0.02      0.04       109\n              Cardiomegaly       0.40      0.06      0.10        68\n              Lung Opacity       0.00      0.00      0.00       126\n               Lung Lesion       0.00      0.00      0.00         1\n                     Edema       0.38      0.18      0.24        45\n             Consolidation       0.00      0.00      0.00        33\n                 Pneumonia       0.00      0.00      0.00         8\n               Atelectasis       0.00      0.00      0.00        80\n              Pneumothorax       0.00      0.00      0.00         8\n          Pleural Effusion       0.32      0.10      0.16        67\n             Pleural Other       0.00      0.00      0.00         1\n                  Fracture       0.00      0.00      0.00         0\n\n                 micro avg       0.24      0.04      0.07       546\n                 m

In [23]:
bce_chexpert_exp.save()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /srv/idp-radio-1/models/InceptionV3_Chexpert_CWBCE_L1Normed_E3_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_5LR1_LF1_SGD_Upsampled/InceptionV3_Chexpert_CWBCE_L1Normed_E3_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_5LR1_LF1_SGD_Upsampled_20200825-003417/assets


'b57e5750-e66a-11ea-bb5e-0242ac110005'

In [23]:
compute_class_weight(bce_chexpert_exp.benchmark.traingen)

(<tf.Tensor: shape=(12,), dtype=float32, numpy=
 array([1.0471137 , 0.86157745, 0.2140513 , 1.2290254 , 0.3829084 ,
        1.5654458 , 0.93557644, 0.34771097, 1.1680024 , 0.2783919 ,
        1.1709423 , 0.7992542 ], dtype=float32)>,
 <tf.Tensor: shape=(12,), dtype=float32, numpy=
 array([1.0471137 , 0.86157745, 0.2140513 , 1.2290254 , 0.3829084 ,
        1.5654458 , 0.93557644, 0.34771097, 1.1680024 , 0.2783919 ,
        1.1709423 , 0.7992542 ], dtype=float32)>,
 <tf.Tensor: shape=(12,), dtype=float32, numpy=
 array([0.7336834 , 0.75095767, 1.2557218 , 0.722264  , 0.90054774,
        0.7086204 , 0.7431472 , 0.93446344, 0.7256606 , 1.0406255 ,
        0.7254881 , 0.7588202 ], dtype=float32)>)

In [ ]:
trained_weights = bce_chexpert_exp.model.get_weights()

In [ ]:
images, labels = bce_chexpert_exp.benchmark.traingen[0]
len(images)

In [ ]:
preds = bce_chexpert_exp.model(images, training=True)
preds.op

In [ ]:
bce_chexpert_exp.model.summary(line_length=150)

In [ ]:
index = -1
trained_weights[index].shape, trained_weights[index], np.isnan(trained_weights[index]).any()

In [ ]:
nan_layers = [i for i in range(len(trained_weights)) if np.isnan(trained_weights[i]).any()] 
#nan_layers   

In [ ]:
mean_weights = {i:np.abs(trained_weights[i]).mean() for i in range(len(trained_weights))}
#mean_weights

In [ ]:
{i:trained_weights[i].shape for i in range(len(trained_weights))}

In [ ]:
for i in range(191, 195):
    print(i, trained_weights[i].shape, trained_weights[i])

In [ ]:
#chexpert_exp.save()

In [ ]:
from datetime import datetime
model_filename = chexpert_exp.model_name + "_" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_filename_tf = model_filename + ".tf"
model_filename_h5 = model_filename + ".h5"
model_filename

In [ ]:
folderpath = Path(os.getcwd()) / 'models' / chexpert_exp.model_name
path = folderpath / model_filename
path

In [ ]:
# make sure path exists, ceate one if necessary
Path(folderpath).mkdir(parents=True, exist_ok=True)
chexpert_exp.model.save(path, save_format="tf")

In [ ]:
# make sure path exists, ceate one if necessary
Path(folderpath).mkdir(parents=True, exist_ok=True)
chexpert_exp.model.save(folderpath / model_filename_h5, save_format="h5")

In [ ]:
!pwd

In [ ]:
folderpath / model_filename_h5

In [ ]:
chexpert_exp.save()

In [ ]:
chexpert_exp.evaluate()

In [ ]:
testgen = chexpert_benchmarks["BCE_E1_B32_C0_N5_D256_DS0595_savetest7"].testgen

In [ ]:
preds = chexpert_exp.model.predict(testgen, steps=len(testgen), verbose=1)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import models
import json
from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from pathlib import Path
from keras.utils.generic_utils import get_custom_objects
from src.metrics.losses import WeightedBinaryCrossentropy, compute_class_weight


groundtruth_label = testgen.get_labels_nonan()
thresholds = [0.5]
for t in thresholds:
    print('current threshold: ', t)
    predictions_bool = (preds >= t)
    y_pred = np.array(predictions_bool, dtype=int)

    # report = classification_report(groundtruth_label, y_pred, target_names=list(CHEXPERT_BENCHMARKS['WBCE_E5_B32_C0'].label_columns))
    # print('sklearn report: ', report)
    
    pre = Precision(thresholds=t)
    pre.update_state(groundtruth_label, predictions_bool)
    print('TF Precision: ', pre.result())
    pre.reset_states()
    
    rec = Recall(thresholds=t)
    rec.update_state(groundtruth_label, predictions_bool)
    print('TF Recall: ', rec.result())
    rec.reset_states()

In [ ]:
report = classification_report(groundtruth_label, y_pred, target_names=chexpert_benchmarks["BCE_E1_B32_C0_N5_D256_DS0595_savetest7"].label_columns)
print('sklearn report: ', report)

In [ ]:
model = tf.keras.models.load_model('models/DenseNet121_Chexpert_BCE_E1_B32_C0_N5_D256_DS0595_savetest7/DenseNet121_Chexpert_BCE_E1_B32_C0_N5_D256_DS0595_savetest7_20200819-125620.h5', compile=False)

In [ ]:
predstwo = model.predict(testgen, steps=len(testgen), verbose=1)

In [ ]:
groundtruth_label = testgen.get_labels_nonan()
predictions_bool = (predstwo >= t)
y_pred = np.array(predictions_bool, dtype=int)
report = classification_report(groundtruth_label, y_pred, target_names=chexpert_benchmarks["BCE_E1_B32_C0_N5_D256_DS0595_savetest7"].label_columns)
print('sklearn report: ', report)

In [ ]:
"test.h5".replace(".h5", "")

In [ ]:
# model_guendel_chestxray14 = densenet(classes=len(CHESTXRAY14_COLUMNS))
model_guendel_chexpert = densenet(classes=len(CHEXPERT_COLUMNS))

#experiment_guendel_chestxray14 = Experiment(chestxray14_benchmark, model_guendel_chestxray14)
experiment_guendel_chexpert = Experiment(CHEXPERT_BENCHMARKS["WBCE_E10_B32"], model_guendel_chexpert, model_name="test_WBCE_32")



In [ ]:
#experiment_guendel_chestxray14_result =  experiment_guendel_chestxray14.run()

In [ ]:
experiment_guendel_chexpert_result =  experiment_guendel_chexpert.run()

In [ ]:
experiment_guendel_chexpert.evaluate()

In [ ]:
experiment_guendel_chexpert.save(upload=False)

In [ ]:
print("done")

In [ ]:
CHEXPERT_BENCHMARKS["WBCE_E10_B32"].as_dict()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.losses import Loss
from tensorflow.python.ops.losses import util as tf_losses_util
from tensorflow.python.framework import tensor_util
from tensorflow.python.framework import constant_op
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import clip_ops
from tensorflow.python.keras import backend_config

In [ ]:
weights_pos = tf.ones((4)) * 20.0
weights_neg = tf.ones((4)) * 0.5
y_true = tf.constant([[0, -1, -1, 1], [-1, 0, -1, -1]], dtype=tf.float32)
y_pred = tf.constant([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]])
loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
loss

In [ ]:
y_true = tf.constant(y_true, dtype=tf.float32)
y_true

In [ ]:
y_pred = tf.constant(y_pred)
y_pred

In [ ]:
if tensor_util.is_tensor(y_pred) and tensor_util.is_tensor(y_true):
    y_pred, y_true = tf_losses_util.squeeze_or_expand_dimensions(
        y_pred, y_true)

epsilon_ = constant_op.constant(
     backend_config.epsilon(), dtype=y_pred.dtype.base_dtype)
y_pred = clip_ops.clip_by_value(y_pred, epsilon_, 1. - epsilon_)

In [ ]:
mask = tf.cast(tf.math.greater_equal(y_true, 0), tf.float32)

# Compute cross entropy from probabilities.
bce_pos = y_true * math_ops.log(y_pred + epsilon_)
bce_neg = (1 - y_true) * math_ops.log(1 - y_pred + epsilon_)

bce_pos = tf.math.multiply_no_nan(bce_pos, mask)
bce_neg = tf.math.multiply_no_nan(bce_neg, mask)

bce = bce_pos * weights_pos + \
    bce_neg * weights_neg

loss = tf.reduce_sum(-bce) / tf.reduce_sum(mask)
loss

In [ ]:
tf.reduce_sum(mask)

In [ ]:
bce_neg

In [ ]:
bce

In [ ]:

from tensorflow.keras import backend as K
